In [ ]:
# By Eashrat Jahan Muniya, Farahdiba Zarin, Ruth Kehali Kassahun
from google.colab import drive
import nibabel as nib
from glob import glob
import os 
import glob

import numpy as np
import matplotlib.pyplot as plt
from nibabel.testing import data_path
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy.linalg import inv, det, norm
from math import sqrt, pi
from functools import partial
from scipy.spatial.distance import dice
import time
import operator
import math
import cv2

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
!pip install --upgrade category_encoders
import category_encoders as ce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 717 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def show_img(img):   
    plt.figure()
    plt.imshow(img.T, cmap='gray',origin='lower')

In [ ]:
def calc_dice(segmented_images, groundtruth_images):
    segData = segmented_images + groundtruth_images
    TP_value = np.amax(segmented_images) + np.amax(groundtruth_images)
    TP = (segData == TP_value).sum()  # found a true positive: segmentation result and groundtruth match(both are positive)
    segData_FP = 2. * segmented_images + groundtruth_images
    segData_FN = segmented_images + 2. * groundtruth_images
    FP = (segData_FP == 2 * np.amax(segmented_images)).sum() # found a false positive: segmentation result and groundtruth mismatch
    FN = (segData_FN == 2 * np.amax(groundtruth_images)).sum() # found a false negative: segmentation result and groundtruth mismatch
    return 2*TP/(2*TP+FP+FN) 

# Loading test masks

In [ ]:
mask_test=[]
for directory_path in sorted( glob.glob("/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask")):
    print(directory_path)
    for img_path in sorted (glob.glob(os.path.join(directory_path, "*.nii.gz"))):
        print(img_path)
        img = nib.load(img_path)
        img_data=img.get_fdata()
        mask_test.append(img_data)

/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1003_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1004_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1005_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1018_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1019_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1023_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1024_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1025_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1038_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1039_1C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_mask/1101_1C.nii.gz
/conten

#Loading test labels 

In [ ]:
label_test=[]
for directory_path in sorted( glob.glob("/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels")):
    print(directory_path)
    for img_path in sorted (glob.glob(os.path.join(directory_path, "*.nii.gz"))):
        print(img_path)
        img = nib.load(img_path)
        img_data=img.get_fdata()
        label_test.append(img_data)

/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1003_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1004_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1005_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1018_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1019_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1023_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1024_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1025_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1038_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_labels/1039_3C.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_label

# Loading test images

In [ ]:
image_test=[]
for directory_path in sorted( glob.glob("/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images")):
    print(directory_path)
    for img_path in sorted (glob.glob(os.path.join(directory_path, "*.nii.gz"))):
        print(img_path)
        img = nib.load(img_path)
        img_data=img.get_fdata()
        image_test.append(img_data)

/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1003.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1004.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1005.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1018.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1019.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1023.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1024.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1025.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1038.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1039.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/testing_images/1101.nii.gz
/content/drive/M

# Tissue Model

In [ ]:
folder_list=["1003", "1004", "1005", "1018", "1019", "1023", "1024", "1025", "1038", "1039", "1101", "1104", "1107", "1110", "1113", "1116", "1119", "1122", "1125", "1128"]
folder_dir="/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3"
lookup_table = pd.read_csv(os.path.join(folder_dir,'tissue_prob' + '.csv'))


seg_tissue_model_list=[]

for i in range(len(image_test)):

  
  image_test[i]=np.multiply(image_test[i],mask_test[i]) # mask application
  image_flatten=image_test[i].copy().flatten() # flattening of image
  image_flatten=pd.DataFrame(image_flatten.astype(int),columns=['intensity value']) # conversion to dataframe

  # addition of probability rows from lookup table if intensity matches

  image_flatten_intensity=image_flatten.merge(lookup_table,left_on='intensity value',right_on='intensity',how='left').drop('intensity',1).fillna(0.0)

  #saving of probabilities

  name='Tissue_Model_Probability_'+folder_list[i]
  if os.path.isfile(os.path.join(folder_dir,'tissue_model_probabilities',name + '.csv')):
    print("File named '%s' already exists." %(name))
    exit()
  else:
    image_flatten_intensity.to_csv(os.path.join(os.path.join(folder_dir,'tissue_model_probabilities',name + '.csv')),mode='a',index=False)
    print("File named '%s' was created." %(name))

  # separating probabilities to apply argmax
  image_flatten_probabilities = image_flatten_intensity.drop(['intensity value'], axis=1)
  image_flatten_intensities_only = image_flatten_intensity['intensity value']

  # count of the number of times equal values appears (if there are multiple zeros for probailities in a single row)
  s = image_flatten_probabilities.eq(image_flatten_probabilities.max(axis=1), axis=0).sum(axis=1)

  # application of argmax
  image_flatten_labels=image_flatten_probabilities.idxmax(axis="columns",skipna=True)
  image_flatten_labels=pd.DataFrame(image_flatten_labels,columns=['labels'])
  image_flatten_probabilities_labels = pd.concat([image_flatten_probabilities,image_flatten_labels], axis=1)

  # assigning the label of background 'BG' to rows that have 3 zeros in a row (meaning the intensity value does not belong to WM, GM, or CSF)
  image_flatten_probabilities_labels['labels'] = image_flatten_probabilities_labels['labels'].mask(s > 2, 'BG')

  # categorical encoding

  # create object of Ordinalencoding

  label_encoder= ce.OrdinalEncoder(cols=['labels'],return_df=True, mapping=[{'col':'labels', 'mapping':{'BG':0,'CSF':1,'GM':3,'WM':2}}])

  # perform label encoding on 'labels' column

  image_flatten_probabilities_labels['labels'] = label_encoder.fit_transform(image_flatten_probabilities_labels['labels'])

  # RESHAPE

  shape_original_image=image_test[i].shape
  shape_original_image_flatten=image_flatten.shape 
  segmented_image=np.zeros(shape_original_image_flatten)

  labels_only=image_flatten_probabilities_labels['labels'].to_numpy() # assigning numerical labels

  segmented_image=labels_only
  Seg=np.reshape(segmented_image,shape_original_image) # reshaping flattened vector of numerical labels to image shape

  imgC=nib.Nifti1Image(Seg,np.eye(4)) # conversion to nifti
  imgC.header.get_xyzt_units()
  imgC.to_filename(os.path.join(folder_dir,'tissue_model_seg_atlas_images','Tissue_Model_Atlas_Seg_'+folder_list[i]+'.nii.gz')) # saving of image

  # WM PROBABILITY MAP
  segmented_image=np.zeros(shape_original_image_flatten)
  segmented_image=image_flatten_probabilities['WM'].to_numpy()
  WM_probability_map=np.reshape(segmented_image,shape_original_image)

  imgC=nib.Nifti1Image(WM_probability_map,np.eye(4))
  imgC.header.get_xyzt_units()
  imgC.to_filename(os.path.join(folder_dir,'tissue_model_probability_maps','WM','Tissue_Model_Probability_Map_WM_'+folder_list[i]+'.nii.gz'))


  # GM PROBABILITY MAP
  segmented_image=np.zeros(shape_original_image_flatten)
  segmented_image=image_flatten_probabilities['GM'].to_numpy()
  GM_probability_map=np.reshape(segmented_image,shape_original_image)

  imgC=nib.Nifti1Image(GM_probability_map,np.eye(4))
  imgC.header.get_xyzt_units()
  imgC.to_filename(os.path.join(folder_dir,'tissue_model_probability_maps','GM','Tissue_Model_Probability_Map_GM_'+folder_list[i]+'.nii.gz'))

  # CSF PROBABILITY MAP
  segmented_image=np.zeros(shape_original_image_flatten)
  segmented_image=image_flatten_probabilities['CSF'].to_numpy()
  CSF_probability_map=np.reshape(segmented_image,shape_original_image)

  imgC=nib.Nifti1Image(CSF_probability_map,np.eye(4))
  imgC.header.get_xyzt_units()
  imgC.to_filename(os.path.join(folder_dir,'tissue_model_probability_maps','CSF','Tissue_Model_Probability_Map_CSF_'+folder_list[i]+'.nii.gz'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


File named 'Tissue_Model_Probability_1003' was created.
File named 'Tissue_Model_Probability_1004' was created.
File named 'Tissue_Model_Probability_1005' was created.
File named 'Tissue_Model_Probability_1018' was created.
File named 'Tissue_Model_Probability_1019' was created.
File named 'Tissue_Model_Probability_1023' was created.
File named 'Tissue_Model_Probability_1024' was created.
File named 'Tissue_Model_Probability_1025' was created.
File named 'Tissue_Model_Probability_1038' was created.
File named 'Tissue_Model_Probability_1039' was created.
File named 'Tissue_Model_Probability_1101' was created.
File named 'Tissue_Model_Probability_1104' was created.
File named 'Tissue_Model_Probability_1107' was created.
File named 'Tissue_Model_Probability_1110' was created.
File named 'Tissue_Model_Probability_1113' was created.
File named 'Tissue_Model_Probability_1116' was created.
File named 'Tissue_Model_Probability_1119' was created.
File named 'Tissue_Model_Probability_1122' was c

# Dice

In [ ]:

seg_img=[]

for directory_path in sorted(glob.glob("/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images")):
    print(directory_path)
    for img_path in glob.glob(os.path.join(directory_path, "*.nii.gz")):
        print(img_path)
        img = nib.load(img_path)
        img_data=img.get_fdata()
        seg_img.append(img_data)
        

/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1003.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1004.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1005.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1018.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1019.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1023.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_model_seg_atlas_images/Tissue_Model_Atlas_Seg_1024.nii.gz
/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/tissue_mode

In [ ]:
# convention labels 
CSF_label = 1
GM_label = 3
WM_label = 2

folder_list=["1003", "1004", "1005", "1018", "1019", "1023", "1024", "1025", "1038", "1039", "1101", "1104", "1107", "1110", "1113", "1116", "1119", "1122", "1125", "1128"]

dice_scores_all=pd.DataFrame(columns=['name','CSF Dice','GM Dice','WM Dice'])

for i in range(20):

  seg_CSF = (seg_img[i] == CSF_label) * 1
  gt_CSF = (label_test[i]== CSF_label) * 1

  seg_GM = (seg_img[i] == GM_label) * 1
  gt_GM = (label_test[i] == GM_label) * 1

  seg_WM = (seg_img[i] == WM_label) * 1
  gt_WM = (label_test[i] == WM_label) * 1

  
  dice_CSF = calc_dice(seg_CSF, gt_CSF)
  dice_GM = calc_dice(seg_GM, gt_GM)
  dice_WM = calc_dice(seg_WM, gt_WM)

  dice_scores={'name':folder_list[i],'CSF Dice':dice_CSF,'GM Dice':dice_GM,'WM Dice':dice_WM}
  dice_scores_all=dice_scores_all.append(dice_scores,ignore_index=True)

print(dice_scores_all)

    name  CSF Dice   GM Dice   WM Dice
0   1003  0.303557  0.830512  0.810838
1   1004  0.600750  0.946359  0.920909
2   1005  0.858138  0.945579  0.919197
3   1018  0.444700  0.913019  0.889330
4   1019  0.398365  0.950403  0.918159
5   1023  0.393945  0.904241  0.879010
6   1024  0.555438  0.931761  0.900932
7   1025  0.846991  0.944773  0.909671
8   1038  0.440875  0.932124  0.910857
9   1039  0.334308  0.942139  0.916382
10  1101  0.676252  0.930337  0.905007
11  1104  0.594084  0.900357  0.862590
12  1107  0.313560  0.714580  0.602438
13  1110  0.558766  0.915449  0.882755
14  1113  0.477339  0.736576  0.710984
15  1116  0.746632  0.937413  0.915115
16  1119  0.795078  0.849852  0.831325
17  1122  0.788992  0.832309  0.813950
18  1125  0.861678  0.916587  0.895219
19  1128  0.691329  0.888672  0.792251


In [ ]:
folder='/content/drive/MyDrive/MAIA/Semester 3/MISA/MISA_MIRA_3/'
name='Tissue_Model_Segmentation_Dice'

if os.path.isfile(os.path.join(folder,name + '.csv')):
  print("File named '%s' already exists." %(name))
  exit()
else:
  dice_scores_all.to_csv(os.path.join(folder,name + '.csv'),mode='a',index=False)
  print("File named '%s' was created." %(name))

File named 'Tissue_Model_Segmentation_Dice' was created.
